# Multi-layer Perceptron Classifier

Source: https://towardsdatascience.com/building-a-speech-emotion-recognizer-using-python-4c1c7c89d713

This model optimizes the log-loss function using LBFGS or stochastic gradient descent.

Uses the notebook "Preprocessing" to preprocess the audio and store the results in a json file.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os, glob, pickle, json 
import time

from sklearn.metrics import recall_score, precision_score, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

from ipynb.fs.full.base_model import BaseModel

from sklearn.experimental import enable_halving_search_cv  
from sklearn.model_selection import HalvingGridSearchCV

In [ ]:
# BASELINE DATASETS
CREMA_D_JSON_FILE_NAME = '../preprocced data/data/baseline/crema-d_preprocessed_data.json'
RAVDESS_JSON_FILE_NAME = '../preprocced data/data/baseline/ravdess_preprocessed_data_V0.1.json'

# AUGMENTED
AUGMENTED_CREMA_D_JSON_FILE_NAME = '../preprocced data/data/augmented_crema-d_preprocessed_data.json'
AUGMENTED_RAVDESS_JSON_FILE_NAME = '../preprocced data/data/augmented_ravdess_preprocessed_data.json'

# Male data
MALE_CREMA_D_JSON_FILE_NAME = '../preprocced data/data/baseline/crema-d_male_preprocessed_data.json'
MALE_RAVDESS_JSON_FILE_NAME = '../preprocced data/data/baseline/ravdess_male_preprocessed_data_V0.1.json'

#Female
FEMALE_CREMA_D_JSON_FILE_NAME = '../preprocced data/data/baseline/crema-d_female_preprocessed_data.json'
FEMALE_RAVDESS_JSON_FILE_NAME = '../preprocced data/data/baseline/ravdess_female_preprocessed_data_V0.1.json'

MODEL_FILENAME = 'MLP_Model.pkl'

In [ ]:
class MLP(BaseModel):    
    """
        RECALL - GRIDSEARCH RESULTS
    """
#     model = MLPClassifier(hidden_layer_sizes=(200),
#                           activation='logistic',
#                           learning_rate='adaptive',
#                           max_iter=400,
#                           solver='sgd',
#                          )
    
    """
        ACCURACY - GRIDSEARCH RESULT
    """
    model = MLPClassifier(hidden_layer_sizes=(100,),
                          activation='logistic',
                          learning_rate='adaptive',
                          max_iter=400,
                          solver='adam',
                         )


#     model = MLPClassifier(hidden_layer_sizes=(300,),
#                           activation='logistic',
#                           learning_rate='constant',
#                           max_iter=10,
#                           alpha=0.003,
#                           solver='adam',
#                           verbose=True
#                          )
    
    @classmethod
    def train(cls, dataset_name):
        train = super().read_dataset(dataset_type='train', dataset_name=dataset_name)
        test = super().read_dataset(dataset_type='test', dataset_name=dataset_name)

        x_train = np.array(train['features'])
        y_train = np.array(train['emotions'])
        x_test = np.array(test['features'])
        y_test = np.array(test['emotions'])

        cls.model.fit(X_train, y_train)

        print(f"\nORIGINAL MODEl: {dataset_name}")
        super().model_accuracy(cls.model, X_train, X_test, y_train, y_test)

        # Augmented data training
        X_train = train_dataset['Augmented']['X']
        y_train = train_dataset['Augmented']['y']
        X_test = test_dataset['Augmented']['X']
        y_test = test_dataset['Augmented']['y']

        cls.model.fit(X_train, y_train)

        print(f"\nAUGMENTED MODEl: {dataset_name}")
        super().model_accuracy(cls.model, X_train, X_test, y_train, y_test)
    
    @classmethod
    def grid_search(cls, dataset_name, is_augmented=False):
        # train dataset
        train_dataset = super().read_dataset(dataset_type='train', dataset_name=dataset_name)

        X_train, y_train = [], []
        
        if is_augmented:
            X_train = train_dataset['Augmented']['X']
            y_train = train_dataset['Augmented']['y']
        else:
            X_train = train_dataset['OriginalData']['X']
            y_train = train_dataset['OriginalData']['y']
        
        # GridSearchCV Train accuracy
        param_grid = [
            {
                'activation' : ['logistic', 'tanh', 'relu'],
                'solver' : ['sgd', 'adam'],
                'hidden_layer_sizes': [
                     (100,), (200,), (300,)
                 ],
                'alpha': [0.0001, 0.05],
                'learning_rate': ['constant', 'adaptive'],
                'max_iter': [100],
            }
        ]
        
        clf = GridSearchCV(model, param_grid, cv=5, scoring=scoring, n_jobs=5) 
        
        start_time = time.perf_counter()
        clf.fit(X_train, y_train)
        end_time = time.perf_counter()
        
        print(f"Duration fitting: {end_time - start_time:04f}")
        
        print("Best parameters set found on development set:")
        print(clf.best_params_)
        print(clf.best_estimator_)
        print(clf.best_score_)
        
        return clf
    
    @classmethod
    def train_default(cls, path, title):
        """
            Training method that uses the normal dataset
        """
        with open(path) as file:
            data = json.load(file)

        x = np.array(data['features'])
        y = np.array(data['emotions'])

        X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

        cls.model.fit(X_train, y_train)
        
        print(f"\nDEFAULT MODEl: {title}")
        super().model_accuracy(cls.model, X_train, X_test, y_train, y_test)

## RAVDESS Accuracy

### Accuracy with train, test and validate

In [ ]:
MLP.train("ravdess")

## CREMA-D Accuracy

### Accuracy with train, test and validate

In [ ]:
MLP.train("crema-d")

# Female and Male accuracy

This chapter shows the accuracy of the original, male and female data of the RAVDESS and CREMA-D dataset.

## RAVDESS

RAVDESS - ORIGINAL DATA: The following code shows the accuracy of the default RAVDESS dataset without augmentation

In [ ]:
MLP.train_default(RAVDESS_JSON_FILE_NAME, "RAVDESS")

RAVDESS - MALE DATA: The following code shows the accuracy of the male RAVDESS dataset without augmentation

In [ ]:
MLP.train_default(MALE_RAVDESS_JSON_FILE_NAME, "RAVDESS")

RAVDESS - FEMALE DATA: The following code shows the accuracy of the female RAVDESS dataset without augmentation

In [ ]:
MLP.train_default(FEMALE_RAVDESS_JSON_FILE_NAME, "RAVDESS")

## CREMA-D

CREMA-D - ORIGINAL DATA: The following code shows the accuracy of the default CREMA-Ddataset without augmentation

In [ ]:
MLP.train_default(CREMA_D_JSON_FILE_NAME, "CREMA-D")

CREMA-D - MALE DATA: The following code shows the accuracy of the male RAVDESS dataset without augmentation

In [ ]:
MLP.train_default(MALE_CREMA_D_JSON_FILE_NAME, "CREMA-D")

CREMA-D - FEMALE DATA: The following code shows the accuracy of the female RAVDESS dataset without augmentation

In [ ]:
MLP.train_default(FEMALE_CREMA_D_JSON_FILE_NAME, "CREMA-D")

# Gridsearch

In [ ]:
# RAVDESS - Not augmented
gridsearch_model = MLP.grid_search('ravdess', is_augmented=False)

In [ ]:
# RAVDESS - augmented
gridsearch_model = MLP.grid_search('ravdess', is_augmented=True)

In [ ]:
# CREMA-D - Not augmented
gridsearch_model = MLP.grid_search('crema-d', is_augmented=False)

In [ ]:
# CREMA-D - augmented
gridsearch_model = MLP.grid_search('crema-d', is_augmented=True)